In [1]:
%cd ..

/volatile/home/Zaccharie/workspace/understanding-unets


In [2]:
# # this just to make sure we are using only on CPU
# import os
# os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [3]:
%load_ext autoreload
%autoreload 2
import os.path as op
import time

from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from keras_tqdm import TQDMNotebookCallback
import tensorflow as tf
from tqdm import tqdm_notebook

from learning_wavelets.datasets import im_dataset_div2k
from learning_wavelets.evaluate import psnr, ssim
from learning_wavelets.keras_utils.filters_cback import NormalizeWeights
from learning_wavelets.keras_utils.image_tboard_cback import TensorBoardImage
from learning_wavelets.keras_utils.normalisation import NormalisationAdjustment
from learning_wavelets.keras_utils.thresholding import SoftThresholding
from learning_wavelets.learned_wavelet import learnlet

Using TensorFlow backend.


In [4]:
tf.random.set_seed(1)

In [5]:
noise_std = 30
batch_size = 8
n_samples_train = 800
im_ds_train = im_dataset_div2k(mode='training', batch_size=batch_size, patch_size=256, noise_std=30, exact_recon=True)
im_ds_val = im_dataset_div2k(mode='validation', batch_size=1, patch_size=256, noise_std=30, exact_recon=True)

In [6]:
thresh = 2*noise_std/255
run_params = {
    'denoising_activation': SoftThresholding(thresh),
    'learnlet_analysis_kwargs':{
        'n_tiling': 256, 
        'mixing_details': False,        
    },
    'n_scales': 5,
    'exact_reconstruction_weight': 1,
}
n_epochs = 250
run_id = f'learnlet_div2k_{noise_std}_{int(time.time())}'
chkpt_path = f'checkpoints/{run_id}' + '-{epoch:02d}.hdf5'
print(run_id)

learnlet_div2k_30_1575052510


In [7]:
chkpt_cback = ModelCheckpoint(chkpt_path, period=n_epochs, save_weights_only=False)
log_dir = op.join('logs', run_id)
tboard_cback = TensorBoard(
    log_dir=log_dir, 
    histogram_freq=0, 
    write_graph=True, 
    write_images=False, 
    profile_batch=0,
)
tqdm_cb = TQDMNotebookCallback(metric_format="{name}: {value:e}")
tqdm_cb.on_train_batch_begin = tqdm_cb.on_batch_begin
tqdm_cb.on_train_batch_end = tqdm_cb.on_batch_end
# tqdm_cb = TQDMProgressBar()
(val_noisy, val_gt), _ = next(iter(im_ds_val))
tboard_image_cback = TensorBoardImage(
    log_dir=log_dir + '/images',
    image=val_gt[0:1],
    noisy_image=val_noisy[0:1],
)
norm_cback = NormalisationAdjustment(momentum=0.99, n_pooling=5, exact_recon=True)
norm_cback.on_train_batch_end = norm_cback.on_batch_end

W1129 19:35:10.587675 139705633687296 callbacks.py:863] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.


In [8]:
n_channels = 1
model = learnlet(input_size=(None, None, n_channels), lr=1e-3, **run_params)
print(model.summary(line_length=114))

Model: "model_1"
__________________________________________________________________________________________________________________
Layer (type)                         Output Shape             Param #       Connected to                          
input_1 (InputLayer)                 [(None, None, None, 1)]  0                                                   
__________________________________________________________________________________________________________________
learnlet_analysis (LearnletAnalysis) [(None, None, None, 256) 32050         input_1[0][0]                         
                                                                            input_2[0][0]                         
__________________________________________________________________________________________________________________
normalisation (Normalisation)        (None, None, None, 256)  256           learnlet_analysis[0][0]               
                                                               

In [9]:
%%time
model.fit(
    im_ds_train, 
    steps_per_epoch=int(n_samples_train / batch_size), 
#     steps_per_epoch=5, 
    epochs=n_epochs,
    validation_data=im_ds_val,
#     validation_steps=int(validation_split * n_samples_train / batch_size),
    validation_steps=1,
    verbose=0,
    callbacks=[tqdm_cb, tboard_cback, chkpt_cback, norm_cback],
#     callbacks=[tqdm_cb, tboard_cback, chkpt_cback, tboard_image_cback, norm_cback, lrate_cback],
    shuffle=False,
)

W1129 19:57:37.781996 139705633687296 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.574649). Check your callbacks.


W1129 19:59:32.214241 139705633687296 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.437749). Check your callbacks.


W1129 22:38:56.916528 139705633687296 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.503690). Check your callbacks.


W1129 23:02:11.541343 139705633687296 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.548342). Check your callbacks.


W1129 23:03:09.321190 139705633687296 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.505569). Check your callbacks.



CPU times: user 1d 9h 50min 57s, sys: 8h 16min, total: 1d 18h 6min 58s
Wall time: 4h 1min 42s


In [10]:
# %%time
# # overfitting trials
# data = next(iter(im_ds_train))
# val_data = next(iter(im_ds_val))
# model.fit(
#     x=data[0], 
#     y=data[1], 
# #     validation_data=val_data, 
#     batch_size=1, 
# #     callbacks=[tqdm_cb, tboard_cback, tboard_image_cback, norm_cback,],
#     callbacks=[tqdm_cb, tboard_cback, norm_cback,],
#     epochs=200, 
#     verbose=2, 
#     shuffle=False,
# )
# # print('Original metrics')
# # print(psnr(data[0].numpy(), data[1].numpy()))
# # print(ssim(data[0].numpy(), data[1].numpy()))

In [11]:
%matplotlib nbagg
import matplotlib.pyplot as plt
import numpy as np

im_denoised = model(data[0]).numpy()
im = data[1].numpy()



fig, axs = plt.subplots(1, 2)
axs[0].imshow(np.squeeze(im))
axs[1].imshow(np.squeeze(im_denoised))

NameError: name 'data' is not defined

In [ ]:
stds = np.array(norm_cback.stds_lists)
i_scale = 1
plt.figure()
plt.plot(stds[i_scale])
plt.ylim([0.3, 1.3]);